In [1]:
import plotly.express as px
import allel
import numpy as np
import pandas as pd

def load_vcf(vcf_path, metadata):
    """
    Load VCF and filter poor-quality samples
    """
    
    sampleIDs = metadata.sampleID.to_list()
    
    # load vcf and get genotypes and positions
    vcf = allel.read_vcf(vcf_path, fields='*')
    samples = vcf['samples']
    # keep only samples in qcpass metadata 
    sample_mask = np.isin(vcf['samples'], metadata.sampleID)
    
    # remove low quality samples 
    geno = allel.GenotypeArray(vcf['calldata/GT'])
    geno = geno.compress(sample_mask, axis=1)
    pos = vcf['variants/POS']
    contig = vcf['variants/CHROM']
    indel = vcf['variants/INDEL']
    
    # remove indels 
    geno = geno.compress(~indel, axis=0)
    pos = pos[~indel]
    contig = contig[~indel]
    
    return geno, pos, contig, samples[sample_mask]

def sample_diversity(geno, samples):
    pis = []
    for i, sample in enumerate(samples):
        ac = geno.take([i], axis=1).count_alleles()
        pis.append(allel.sequence_diversity(ac=ac, pos=np.arange(len(pos))))

    return pd.DataFrame({'sampleID':samples, 'pi':pis})

def cohort_diversity(geno, samples, metadata, cohort_col):
    cohs = metadata[cohort_col].unique()
    coh_idxs = {loc:np.where(metadata[cohort_col] == loc)[0] for loc in cohs}

    pis = []
    for coh in cohs:
        ac = geno.take(coh_idxs[coh], axis=1).count_alleles()
        pis.append(allel.sequence_diversity(ac=ac, pos=np.arange(len(pos))))   

    return pd.DataFrame({'cohort':cohs, 'pi':pis}) 

In [ ]:
dataset = 'vigg-01'
metadata_path = "../../results/config/metadata.qcpass.tsv"
cohort_cols = 'location,taxon'
vcf_path = "../../results/vcfs/amplicons/ampseq-vigg-01.annot.vcf"
wkdir = "../.."

### Estimating genetic diversity

This page calculates genetic diversity in individuals and cohorts. 

Calculating genetic diversity from ag-vampir amplicons is tricky because there are so many Vgsc amplicons, results will be biased by the presence of selective sweeps, and aims. 

In [ ]:
cohort_cols = cohort_cols.split(",")

# load metadata
if metadata_path.endswith('.xlsx'):
    metadata = pd.read_excel(metadata_path, engine='openpyxl')
elif metadata_path.endswith('.tsv'):
    metadata = pd.read_csv(metadata_path, sep="\t")
elif metadata_path.endswith('.csv'):
    metadata = pd.read_csv(metadata_path, sep=",")
else:
    raise ValueError("Metadata file must be .xlsx or .csv")

geno, pos, contig, samples = load_vcf(vcf_path, metadata)

#### By cohort

In [ ]:
for coh in cohort_cols:
    cohort_pi_df = cohort_diversity(
        geno=geno, 
        samples=samples, 
        metadata=metadata, 
        cohort_col=coh
    )
    cohort_pi_df.to_csv(f"{wkdir}/results/genetic-diversity/{coh}.pi.tsv", sep="\t")
    px.bar(cohort_pi_df, x='cohort', y='pi', template='simple_white', width=600, height=400)

In [ ]:
sample_pi_df = sample_diversity(geno, samples)
sample_pi_df.to_csv(f"{wkdir}/results/genetic-diversity/samples.pi.tsv", sep="\t")